In [1]:
import pandas as pd
import re
import emoji
#!pip install regex
import regex as rex
import chardet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import glob
import seaborn as sns
from category_encoders import OneHotEncoder
from IPython.display import VimeoVideo
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted
from forex_python.converter import CurrencyRates
import chardet
import re
import os

#Untuk menyatukan beberapa file CSV ke dalam satu file
file_pattern = r"myDataPattern*.csv" #Pola nama file

# Use glob to find all CSV files matching the pattern
file_list = glob.glob(file_pattern)

# Initialize an empty list to hold the DataFrames
dfs = []

# Iterate over each file, read it into a DataFrame, and append to the list
for file in file_list:
    dfs.append(pd.read_csv(file))

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Now combined_df contains all data from the CSV files
print(combined_df.head())

df = combined_df

df.select_dtypes('object').nunique()



                          timestamp           author comment super_sticker  \
0  2024-06-12T12:33:46.500257+00:00        tron(とろん)     NaN           NaN   
1  2024-06-12T12:33:12.921018+00:00          Andante     NaN           NaN   
2  2024-06-12T12:33:06.152873+00:00          オレンジタオル     NaN           NaN   
3  2024-06-12T12:32:42.753364+00:00  あくあ組　親衛隊445番　プギ     NaN           NaN   
4  2024-06-12T12:31:53.635906+00:00           キリコ・QB     NaN           NaN   

                          super_chat  is_member super_chat_amount new_member  \
0                                NaN       True               NaN        NaN   
1                                NaN       True               NaN        NaN   
2  自分が王国民になるきっかけになった神曲です。ありがとうございます！      False       JPY 1600.00        NaN   
3                    応援してます！あくたんサイコー       True       JPY 3200.00        NaN   
4        ゆかりーん！ ゆ、ゆーっ、ユアアーッ!! ユアーッ!!      False       JPY 1000.00        NaN   

  membership_level member_since  
0              N

timestamp            61282
author                3818
comment              21270
super_sticker           12
super_chat              89
super_chat_amount       27
new_member              25
membership_level         1
member_since             1
dtype: int64

In [2]:
def clean_comments(text):
    if pd.isna(text):
        return text
    # Define regex patterns
    emoji_pattern = re.compile(
        "["  
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251" 
        "]+",
        flags=re.UNICODE
    )
    non_japanese_pattern = re.compile(r'[^\u3040-\u30FF\u3400-\u4DBF\u4E00-\u9FFF]+')  # Match non-Japanese characters
    
    # Remove emojis and non-Japanese words
    cleaned_text = emoji_pattern.sub(r'', text)
    cleaned_text = non_japanese_pattern.sub(r'', cleaned_text)
    return cleaned_text

In [3]:
df['super_chat_amount'].notnull().sum()

89

In [4]:
df.fillna(np.nan, inplace=True)
# Define a function to clean the text
def clean_text(text):
    if pd.notna(text):
        # Use regular expression to remove all substrings that match the pattern
        cleaned_text = re.sub(r':_.*?:', '', text)
        cleaned_text = re.sub(r':[^:]*:', '', text)
        cleaned_text = re.sub(r'[♪♪♪♪]+', '', cleaned_text)
        # Remove any numbers (digits)
        cleaned_text = re.sub(r'\d+', '', cleaned_text)
        return cleaned_text.strip()  # Strip leading/trailing whitespace
    else:
        return text

# Apply the cleaning function to the 'comment' column
df['comment'] = df['comment'].apply(clean_text)

# Print the DataFrame with the cleaned text column
print(df['comment'].head(10))

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8       
9    可愛い
Name: comment, dtype: object


In [5]:
print(f"Number of files in file_list: {len(file_list)}")

Number of files in file_list: 62


In [6]:
df[["Currency_Name", "Currency_Value"]] = df['super_chat_amount'].str.split(" ", expand=True)
df.head(10)

,timestamp,author,comment,super_sticker,super_chat,is_member,super_chat_amount,new_member,membership_level,member_since,Currency_Name,Currency_Value
0,2024-06-12T12:33:46.500257+00:00,tron(とろん),NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-06-12T12:33:12.921018+00:00,Andante,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-06-12T12:33:06.152873+00:00,オレンジタオル,NaN,NaN,自分が王国民になるきっかけになった神曲です。ありがとうございます！,False,JPY 1600.00,NaN,NaN,NaN,JPY,1600.00
3,2024-06-12T12:32:42.753364+00:00,あくあ組 親衛隊445番 プギ,NaN,NaN,応援してます！あくたんサイコー,True,JPY 3200.00,NaN,NaN,NaN,JPY,3200.00
4,2024-06-12T12:31:53.635906+00:00,キリコ・QB,NaN,NaN,ゆかりーん！ ゆ、ゆーっ、ユアアーッ!! ユアーッ!!,False,JPY 1000.00,NaN,NaN,NaN,JPY,1000.00
5,2024-06-12T12:31:20.864966+00:00,うすはな,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-06-12T12:03:45.952131+00:00,qfeuille3⚓🥐,NaN,NaN,:_aquaむーちゃん::_aquaサイリウム::_aquaキラキラ::_aquaおんぷ::...,True,SGD 50.00,NaN,NaN,NaN,SGD,50.00
7,2024-06-12T11:49:12.96318+00:00,林志南,NaN,NaN,老人会歌枠&告知ありがとう､学校から帰ってきたら深夜までアニメを見た頃､お金ないのでグッズ買...,True,TWD 1500.00,NaN,NaN,NaN,TWD,1500.00
8,2024-06-12T12:34:01.62595+00:00,コワカロッゾ,,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-06-12T12:34:01.666953+00:00,大好き黒ダル(たまにゲーム実況者),可愛い,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
print(type(df))
print(df.info())

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61282 entries, 0 to 61281
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   timestamp          61282 non-null  object
 1   author             61095 non-null  object
 2   comment            61122 non-null  object
 3   super_sticker      12 non-null     object
 4   super_chat         89 non-null     object
 5   is_member          61282 non-null  bool  
 6   super_chat_amount  89 non-null     object
 7   new_member         25 non-null     object
 8   membership_level   25 non-null     object
 9   member_since       25 non-null     object
 10  Currency_Name      89 non-null     object
 11  Currency_Value     89 non-null     object
dtypes: bool(1), object(11)
memory usage: 5.2+ MB
None


In [8]:
df.select_dtypes('object').nunique()
df['Currency_Name'].unique()

array([nan, 'JPY', 'SGD', 'TWD', 'HKD', 'MYR', 'AUD', 'ARS', 'KRW', 'USD',
       'THB'], dtype=object)

In [9]:
df["Currency_Value"] = df.Currency_Value.astype(float)

In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61282 entries, 0 to 61281
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   timestamp          61282 non-null  object 
 1   author             61095 non-null  object 
 2   comment            61122 non-null  object 
 3   super_sticker      12 non-null     object 
 4   super_chat         89 non-null     object 
 5   is_member          61282 non-null  bool   
 6   super_chat_amount  89 non-null     object 
 7   new_member         25 non-null     object 
 8   membership_level   25 non-null     object 
 9   member_since       25 non-null     object 
 10  Currency_Name      89 non-null     object 
 11  Currency_Value     89 non-null     float64
dtypes: bool(1), float64(1), object(10)
memory usage: 5.2+ MB
None


In [11]:
currency_symbols_mapping = {
    '$': 'USD',    # United States Dollar
    '€': 'EUR',    # Euro
    '¥': 'JPY',    # Japanese Yen
    '£': 'GBP',    # British Pound Sterling
    '₹': 'INR',    # Indian Rupee
    '₽': 'RUB',    # Russian Ruble
    '₿': 'BTC',    # Bitcoin (digital currency)
    '฿': 'THB',    # Thai Baht
    '₺': 'TRY',    # Turkish Lira
    '₴': 'UAH',    # Ukrainian Hryvnia
    '₡': 'CRC',    # Costa Rican Colón
    '₦': 'NGN',    # Nigerian Naira
    '₫': 'VND',    # Vietnamese Dong
    '₼': 'AZN',    # Azerbaijani Manat
    '₪': 'ILS',    # Israeli New Shekel
    '₸': 'KZT',    # Kazakhstani Tenge
    '₭': 'LAK',    # Lao Kip
    '₮': 'MNT',    # Mongolian Tugrik
    '₱': 'PHP',    # Philippine Peso
    '₨': 'PKR',    # Pakistani Rupee
    '₰': 'XAG',    # Silver Ounce (troy ounce of silver)
    '₱': 'CLP',    # Chilean Peso
    '₲': 'PYG',    # Paraguayan Guarani
    '₵': 'GHS',    # Ghanaian Cedi
    '₳': 'ARA',    # Argentine Austral
    '₴': 'GRD',    # Greek Drachma
    '₩': 'KRW',    # South Korean Won
    '₥': 'SVC',    # Salvadoran Colón
    '₦': 'NGN',    # Nigerian Naira
    '₧': 'ESP',    # Spanish Peseta
    '₰': 'GMD',    # Gambian Dalasi
    '₸': 'KZT',    # Kazakhstani Tenge
    '₮': 'MNT',    # Mongolian Tugrik
    '₺': 'TRY',    # Turkish Lira
}


In [12]:
def extract_amount_currency(super_sticker):
    if pd.isna(super_sticker):
        return np.nan, np.nan
    else:
        # Find the index of the first digit in the super_sticker string
        first_digit_index = next((i for i, c in enumerate(super_sticker) if c.isdigit()), None)
        if first_digit_index is not None:
            # Extract currency and amount based on the position of the first digit
            currency_symbol = super_sticker[:first_digit_index]
            amount_str = ''
            for i in range(first_digit_index, len(super_sticker)):
                if super_sticker[i].isdigit() or super_sticker[i] == '.' or super_sticker[i] == ',':
                    if super_sticker[i] != ',':
                        amount_str += super_sticker[i]
                else:
                    break
            amount = float(amount_str)
            # Map currency symbol to currency name if applicable
            currency = currency_symbols_mapping.get(currency_symbol, currency_symbol)
            return amount, currency
        else:
            return np.nan, np.nan

df['super_sticker_amount'], df['super_sticker_currency'] = zip(*df['super_sticker'].apply(extract_amount_currency))
print(df[['super_sticker', 'super_sticker_amount', 'super_sticker_currency']].head())

  super_sticker  super_sticker_amount super_sticker_currency
0           NaN                   NaN                    NaN
1           NaN                   NaN                    NaN
2           NaN                   NaN                    NaN
3           NaN                   NaN                    NaN
4           NaN                   NaN                    NaN


In [13]:
df['super_sticker_currency'].unique()

array([nan, 'JPY', 'USD', 'NT$'], dtype=object)

In [14]:
exchange_rates = {
    'JPY': 0.0064,  # Example exchange rate for JPY to USD
    'TWD': 0.031,   # Example exchange rate for TWD to USD
    'SGD': 0.74     # Example exchange rate for SGD to USD
}

# Function to convert amount to USD based on currency name
def convert_to_usd(row):
    currency = row['Currency_Name']
    amount = row['Currency_Value']
    if currency in exchange_rates:
        exchange_rate = exchange_rates[currency]
        amount_usd = float(amount) * exchange_rate
        return amount_usd
    else:
        return None
# Function to convert amount to USD based on currency name
def convert_sticker_to_usd(row):
    currency = row['super_sticker_currency']
    amount = row['super_sticker_amount']
    if currency in exchange_rates:
        exchange_rate = exchange_rates[currency]
        amount_usd = float(amount) * exchange_rate
        return amount_usd
    else:
        return None

# Apply conversion function to create new column 'super_chat_amount_usd'
df['super_chat_amount_usd'] = df.apply(convert_to_usd, axis=1)
df['super_sticker_amount_usd'] = df.apply(convert_sticker_to_usd, axis=1)

In [15]:
print(df[['super_sticker', 'super_sticker_amount', 'Currency_Name','super_sticker_currency', 'super_sticker_amount_usd']].head())

  super_sticker  super_sticker_amount Currency_Name super_sticker_currency  \
0           NaN                   NaN           NaN                    NaN   
1           NaN                   NaN           NaN                    NaN   
2           NaN                   NaN           JPY                    NaN   
3           NaN                   NaN           JPY                    NaN   
4           NaN                   NaN           JPY                    NaN   

   super_sticker_amount_usd  
0                       NaN  
1                       NaN  
2                       NaN  
3                       NaN  
4                       NaN  


In [16]:
df

,timestamp,author,comment,super_sticker,super_chat,is_member,super_chat_amount,new_member,membership_level,member_since,Currency_Name,Currency_Value,super_sticker_amount,super_sticker_currency,super_chat_amount_usd,super_sticker_amount_usd
0,2024-06-12T12:33:46.500257+00:00,tron(とろん),NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-06-12T12:33:12.921018+00:00,Andante,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-06-12T12:33:06.152873+00:00,オレンジタオル,NaN,NaN,自分が王国民になるきっかけになった神曲です。ありがとうございます！,False,JPY 1600.00,NaN,NaN,NaN,JPY,1600.0,NaN,NaN,10.24,NaN
3,2024-06-12T12:32:42.753364+00:00,あくあ組 親衛隊445番 プギ,NaN,NaN,応援してます！あくたんサイコー,True,JPY 3200.00,NaN,NaN,NaN,JPY,3200.0,NaN,NaN,20.48,NaN
4,2024-06-12T12:31:53.635906+00:00,キリコ・QB,NaN,NaN,ゆかりーん！ ゆ、ゆーっ、ユアアーッ!! ユアーッ!!,False,JPY 1000.00,NaN,NaN,NaN,JPY,1000.0,NaN,NaN,6.40,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61277,2024-06-12T12:48:41.501643+00:00,Yuki,待ってました！！,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61278,2024-06-12T12:48:41.613042+00:00,快点完结,,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61279,2024-06-12T12:48:41.68796+00:00,三ヶ月 心,は～い,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61280,2024-06-12T12:48:41.703166+00:00,業火,💕💕💕,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
def extract_message(super_sticker):
    if pd.notna(super_sticker):
        message_start = super_sticker.find(':')
        if message_start != -1:  # If ":" is found
            return super_sticker[message_start + 1:].strip()  # Extract message after ":"
    return None  # Return None if ":" is not found or super_sticker is NaN

# Create new column 'message'
df['message'] = df['super_sticker'].apply(extract_message)

# Print DataFrame with the new column
print(df)

                              timestamp           author   comment  \
0      2024-06-12T12:33:46.500257+00:00        tron(とろん)       NaN   
1      2024-06-12T12:33:12.921018+00:00          Andante       NaN   
2      2024-06-12T12:33:06.152873+00:00          オレンジタオル       NaN   
3      2024-06-12T12:32:42.753364+00:00  あくあ組　親衛隊445番　プギ       NaN   
4      2024-06-12T12:31:53.635906+00:00           キリコ・QB       NaN   
...                                 ...              ...       ...   
61277  2024-06-12T12:48:41.501643+00:00             Yuki  待ってました！！   
61278  2024-06-12T12:48:41.613042+00:00             快点完结             
61279   2024-06-12T12:48:41.68796+00:00            三ヶ月　心       は～い   
61280  2024-06-12T12:48:41.703166+00:00               業火       💕💕💕   
61281  2024-06-12T12:48:41.742679+00:00               脳C    告知わくわく   

      super_sticker                         super_chat  is_member  \
0               NaN                                NaN       True   
1               NaN  

In [18]:
df.info

<bound method DataFrame.info of                               timestamp           author   comment  \
0      2024-06-12T12:33:46.500257+00:00        tron(とろん)       NaN   
1      2024-06-12T12:33:12.921018+00:00          Andante       NaN   
2      2024-06-12T12:33:06.152873+00:00          オレンジタオル       NaN   
3      2024-06-12T12:32:42.753364+00:00  あくあ組　親衛隊445番　プギ       NaN   
4      2024-06-12T12:31:53.635906+00:00           キリコ・QB       NaN   
...                                 ...              ...       ...   
61277  2024-06-12T12:48:41.501643+00:00             Yuki  待ってました！！   
61278  2024-06-12T12:48:41.613042+00:00             快点完结             
61279   2024-06-12T12:48:41.68796+00:00            三ヶ月　心       は～い   
61280  2024-06-12T12:48:41.703166+00:00               業火       💕💕💕   
61281  2024-06-12T12:48:41.742679+00:00               脳C    告知わくわく   

      super_sticker                         super_chat  is_member  \
0               NaN                                NaN    

In [19]:
def remove_emoji(string):
    # Custom regex to match emojis
    emoji_pattern = re.compile(
        "["u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

# Handle NaN values by filling them with an empty string
df['comment'] = df['comment'].fillna('')

# Apply the function to the 'comment' column
df['comments_clean'] = df['comment'].apply(remove_emoji)

print(df)

                              timestamp           author   comment  \
0      2024-06-12T12:33:46.500257+00:00        tron(とろん)             
1      2024-06-12T12:33:12.921018+00:00          Andante             
2      2024-06-12T12:33:06.152873+00:00          オレンジタオル             
3      2024-06-12T12:32:42.753364+00:00  あくあ組　親衛隊445番　プギ             
4      2024-06-12T12:31:53.635906+00:00           キリコ・QB             
...                                 ...              ...       ...   
61277  2024-06-12T12:48:41.501643+00:00             Yuki  待ってました！！   
61278  2024-06-12T12:48:41.613042+00:00             快点完结             
61279   2024-06-12T12:48:41.68796+00:00            三ヶ月　心       は～い   
61280  2024-06-12T12:48:41.703166+00:00               業火       💕💕💕   
61281  2024-06-12T12:48:41.742679+00:00               脳C    告知わくわく   

      super_sticker                         super_chat  is_member  \
0               NaN                                NaN       True   
1               NaN  

In [20]:
def remove_currency(string):
    # Remove common currency symbols and thousands separators
    string = re.sub(r'[$€,¥]', '', string)
    string = re.sub(r',', '', string)
    return string

In [21]:
df['comments_clean'] = df['comment'].apply(remove_emoji)
df['comments_clean'] = df['comments_clean'].apply(clean_comments)

df['comments_clean']
print(df)

                              timestamp           author   comment  \
0      2024-06-12T12:33:46.500257+00:00        tron(とろん)             
1      2024-06-12T12:33:12.921018+00:00          Andante             
2      2024-06-12T12:33:06.152873+00:00          オレンジタオル             
3      2024-06-12T12:32:42.753364+00:00  あくあ組　親衛隊445番　プギ             
4      2024-06-12T12:31:53.635906+00:00           キリコ・QB             
...                                 ...              ...       ...   
61277  2024-06-12T12:48:41.501643+00:00             Yuki  待ってました！！   
61278  2024-06-12T12:48:41.613042+00:00             快点完结             
61279   2024-06-12T12:48:41.68796+00:00            三ヶ月　心       は～い   
61280  2024-06-12T12:48:41.703166+00:00               業火       💕💕💕   
61281  2024-06-12T12:48:41.742679+00:00               脳C    告知わくわく   

      super_sticker                         super_chat  is_member  \
0               NaN                                NaN       True   
1               NaN  

In [22]:
def remove_emoji(string):
    if isinstance(string, str):
        return emoji.replace_emoji(string, replace='')
    return string  # if not string, return as is

In [23]:
def remove_currency(string):
    if isinstance(string, str):
        string = re.sub(r'[$€,¥]', '', string)
        string = re.sub(r',', '', string)
        return string
    return string  # if not string, return as is

In [24]:
# Apply the remove_emoji function to the 'comments' column
df['comments_clean'] = df['comment'].apply(remove_emoji)

df['superchat_clean'] = df['super_chat'].apply(remove_emoji)

df = df.drop(columns='super_chat')

df.head(10)

,timestamp,author,comment,super_sticker,is_member,super_chat_amount,new_member,membership_level,member_since,Currency_Name,Currency_Value,super_sticker_amount,super_sticker_currency,super_chat_amount_usd,super_sticker_amount_usd,message,comments_clean,superchat_clean
0,2024-06-12T12:33:46.500257+00:00,tron(とろん),,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,NaN
1,2024-06-12T12:33:12.921018+00:00,Andante,,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,NaN
2,2024-06-12T12:33:06.152873+00:00,オレンジタオル,,NaN,False,JPY 1600.00,NaN,NaN,NaN,JPY,1600.0,NaN,NaN,10.24,NaN,None,,自分が王国民になるきっかけになった神曲です。ありがとうございます！
3,2024-06-12T12:32:42.753364+00:00,あくあ組 親衛隊445番 プギ,,NaN,True,JPY 3200.00,NaN,NaN,NaN,JPY,3200.0,NaN,NaN,20.48,NaN,None,,応援してます！あくたんサイコー
4,2024-06-12T12:31:53.635906+00:00,キリコ・QB,,NaN,False,JPY 1000.00,NaN,NaN,NaN,JPY,1000.0,NaN,NaN,6.40,NaN,None,,ゆかりーん！ ゆ、ゆーっ、ユアアーッ!! ユアーッ!!
5,2024-06-12T12:31:20.864966+00:00,うすはな,,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,NaN
6,2024-06-12T12:03:45.952131+00:00,qfeuille3⚓🥐,,NaN,True,SGD 50.00,NaN,NaN,NaN,SGD,50.0,NaN,NaN,37.00,NaN,None,,:_aquaむーちゃん::_aquaサイリウム::_aquaキラキラ::_aquaおんぷ::...
7,2024-06-12T11:49:12.96318+00:00,林志南,,NaN,True,TWD 1500.00,NaN,NaN,NaN,TWD,1500.0,NaN,NaN,46.50,NaN,None,,老人会歌枠&告知ありがとう､学校から帰ってきたら深夜までアニメを見た頃､お金ないのでグッズ買...
8,2024-06-12T12:34:01.62595+00:00,コワカロッゾ,,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,NaN
9,2024-06-12T12:34:01.666953+00:00,大好き黒ダル(たまにゲーム実況者),可愛い,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,可愛い,NaN


In [25]:
def filter_japanese(text):
    if isinstance(text, str):
        # Regex pattern to match Japanese characters (Hiragana, Katakana, Kanji)
        pattern = rex.compile(r'[^\p{Script=Hiragana}\p{Script=Katakana}\p{Script=Han}]')
        # Replace non-Japanese characters with an empty string
        return pattern.sub('', text)
    else:
        return text

# Handle NaN values by filling them with an empty string
df['superchat_clean'] = df['superchat_clean'].fillna('')

# Apply the function to the 'superchat_clean' column
df['superchat_clean'] = df['superchat_clean'].apply(filter_japanese)
df

,timestamp,author,comment,super_sticker,is_member,super_chat_amount,new_member,membership_level,member_since,Currency_Name,Currency_Value,super_sticker_amount,super_sticker_currency,super_chat_amount_usd,super_sticker_amount_usd,message,comments_clean,superchat_clean
0,2024-06-12T12:33:46.500257+00:00,tron(とろん),,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,
1,2024-06-12T12:33:12.921018+00:00,Andante,,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,
2,2024-06-12T12:33:06.152873+00:00,オレンジタオル,,NaN,False,JPY 1600.00,NaN,NaN,NaN,JPY,1600.0,NaN,NaN,10.24,NaN,None,,自分が王国民になるきっかけになった神曲ですありがとうございます
3,2024-06-12T12:32:42.753364+00:00,あくあ組 親衛隊445番 プギ,,NaN,True,JPY 3200.00,NaN,NaN,NaN,JPY,3200.0,NaN,NaN,20.48,NaN,None,,応援してますあくたんサイコ
4,2024-06-12T12:31:53.635906+00:00,キリコ・QB,,NaN,False,JPY 1000.00,NaN,NaN,NaN,JPY,1000.0,NaN,NaN,6.40,NaN,None,,ゆかりんゆゆっユアアッユアッ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61277,2024-06-12T12:48:41.501643+00:00,Yuki,待ってました！！,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,待ってました！！,
61278,2024-06-12T12:48:41.613042+00:00,快点完结,,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,
61279,2024-06-12T12:48:41.68796+00:00,三ヶ月 心,は～い,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,は～い,
61280,2024-06-12T12:48:41.703166+00:00,業火,💕💕💕,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,


In [26]:
df['comments_clean']= df['comments_clean'].apply(filter_japanese)

In [27]:
df = df.drop(columns='comment')

In [28]:
df

,timestamp,author,super_sticker,is_member,super_chat_amount,new_member,membership_level,member_since,Currency_Name,Currency_Value,super_sticker_amount,super_sticker_currency,super_chat_amount_usd,super_sticker_amount_usd,message,comments_clean,superchat_clean
0,2024-06-12T12:33:46.500257+00:00,tron(とろん),NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,
1,2024-06-12T12:33:12.921018+00:00,Andante,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,
2,2024-06-12T12:33:06.152873+00:00,オレンジタオル,NaN,False,JPY 1600.00,NaN,NaN,NaN,JPY,1600.0,NaN,NaN,10.24,NaN,None,,自分が王国民になるきっかけになった神曲ですありがとうございます
3,2024-06-12T12:32:42.753364+00:00,あくあ組 親衛隊445番 プギ,NaN,True,JPY 3200.00,NaN,NaN,NaN,JPY,3200.0,NaN,NaN,20.48,NaN,None,,応援してますあくたんサイコ
4,2024-06-12T12:31:53.635906+00:00,キリコ・QB,NaN,False,JPY 1000.00,NaN,NaN,NaN,JPY,1000.0,NaN,NaN,6.40,NaN,None,,ゆかりんゆゆっユアアッユアッ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61277,2024-06-12T12:48:41.501643+00:00,Yuki,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,待ってました,
61278,2024-06-12T12:48:41.613042+00:00,快点完结,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,
61279,2024-06-12T12:48:41.68796+00:00,三ヶ月 心,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,はい,
61280,2024-06-12T12:48:41.703166+00:00,業火,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,


In [34]:
columns_to_check = [
    'super_sticker', 'super_chat_amount', 'Currency_Name', 
    'Currency_Value', 'super_sticker_amount', 'super_chat_amount_usd', 
    'super_sticker_amount_usd', 'message', 'comments_clean', 'superchat_clean'
]

# Filter rows where all specified columns contain an empty string or None
filtered_df = df[~df[columns_to_check].apply(lambda x: x.fillna('').eq('').all(), axis=1)]

filtered_df.shape

(39615, 17)

In [ ]:
filtered

In [35]:
# Remove '_part' from the filename
filename_without_part = os.path.basename(first_file).replace('_part', '_')

# Define the output directory and filename for the filtered data
output_dir = 'CommentBlob'
output_filename = os.path.join(output_dir, f"{filename_without_part}")

# Ensure the output directory exists; create it if it doesn't
os.makedirs(output_dir, exist_ok=True)

# Save the filtered DataFrame to CSV with the updated filename
filtered_df.to_csv(output_filename, index=False)

print(f"Filtered data saved to {output_filename}")

Filtered data saved to CommentBlob\Aqua Ch. 湊あくあ_2024-06-12_1.csv


In [32]:
file_path = output_filename

# Detect the file encoding
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

print(f"Detected encoding: {encoding}")

# Read the CSV file with the detected encoding
filtered_df = pd.read_csv(file_path, encoding=encoding)

# Save the CSV file with UTF-8 BOM encoding
corrected_file_path = file_path.replace(".csv", "_corrected.csv")
filtered_df.to_csv(corrected_file_path, index=False, encoding='utf-8-sig')

print(f"Corrected file saved to: {corrected_file_path}")

Detected encoding: None
Corrected file saved to: CommentBlob\Aqua Ch. 湊あくあ_2024-06-12_1_corrected.csv
